# 8 Queens

### Introduction

<figure>
<img src="resources/eight_queens_moves.png", width=300 align="right">
    <figcaption></figcaption>
</figure>

The [Eight Queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is a famous puzzle that has been studied extensively in- and outside of computer science. It was first published in the chess magazine _Schach_ in 1848. 

The problem can be formulated as follows: 

_"Place 8 queens on a regular (8x8) chess board such that no queen attacks any other queen."_

A queen in the game of chess can move horizontally, vertically, and diagonally. The puzzle can be solved by hand (and even [Carl Friedrich Gauss](https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss) studied it back in 1850).

The EightQueensState class below, as well as the methods defined, should prove a helpful start for a Genetic Algorithms approach. However, you are welcome to change as little or as much of the code as is useful.

In [9]:
import numpy as np
configuration_costs = {}

class EightQueensState:
    """This class represents a board in the eight queens puzzle"""
    def __init__(self, state=None, n=8):
        """
        :param state: pass in a numpy array of integers to set the state, otherwise will be generated randomly
        :param n: only used if state is not provided, determines size of board (default: 8)
        """
        if state is None:
            self.n = n
            state = np.random.randint(0, n, n)
        else:
            self.n = len(state)
        self.state = state

    @staticmethod
    def copy_replace(state, i, x):
        """This creates a copy of the state (important as numpy arrays are mutable) with column i set to x"""
        new_state = state.copy()
        new_state[i] = x
        return new_state

    @staticmethod
    def range_missing(start, stop, missing):
        """
        This creates a list of numbers with a single value missing
        e.g. range_missing(0, 8, 2) -> [0, 1, 3, 4, 5, 6, 7]
        """
        return list(range(start, missing)) + list(range(missing + 1, stop))
    
    def cost(self):
        """Calculates the number of pairs attacking"""
        "Small change on the cost function to improve the performance"
        s = str(self.state)
        if s not in configuration_costs:
            count = 0
            for i in range(len(self.state) - 1):
                # for each queen, look in columns to the right
                # add one to the count if there is another queen in the same row
                count += (self.state[i] == np.array(self.state[i + 1:])).sum()

                # add one to the count for each queen on the upper or lower diagonal
                upper_diagonal = self.state[i] + np.arange(1, self.n - i)
                lower_diagonal = self.state[i] - np.arange(1, self.n - i)
                count += (np.array(self.state[i + 1:]) == upper_diagonal).sum()
                count += (np.array(self.state[i + 1:]) == lower_diagonal).sum()
            configuration_costs[s] = count
        return configuration_costs[s]
    
    def neighbourhood(self):
        """This generates every state possible by changing a single queen position"""
        neighbourhood = []
        for column in range(self.n):
            for new_position in self.range_missing(0, self.n, self.state[column]):
                new_state = self.copy_replace(self.state, column, new_position)
                neighbourhood.append(EightQueensState(new_state))

        return neighbourhood

    def random_neighbour(self):
        """Generates a single random neighbour state, useful for some algorithms"""
        column = np.random.choice(range(self.n))
        new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
        new_state = self.copy_replace(self.state, column, new_position)
        return EightQueensState(new_state)

    def is_goal(self):
        return self.cost() == 0
    
    def is_not_goal(self):
        """Additional Code, check if it is not goal"""
        count = 0
        for i in range(len(self.state) - 1):
            # for each queen, look in columns to the right
            # add one to the count if there is another queen in the same row
            count += (self.state[i] == np.array(self.state[i + 1:])).sum()
            if count >0 :
                return True
            # add one to the count for each queen on the upper or lower diagonal
            upper_diagonal = self.state[i] + np.arange(1, self.n - i)
            lower_diagonal = self.state[i] - np.arange(1, self.n - i)
            count += (np.array(self.state[i + 1:]) == upper_diagonal).sum()
            count += (np.array(self.state[i + 1:]) == lower_diagonal).sum()
            if count >0 :
                return True
        return False

    def __str__(self):
        if self.is_goal():
            return f"Goal state! {self.state}"
        else:
            return f"{self.state} cost {self.cost()}"

---

### Version 1- Solve the 8 Queens problem using a genetic algorithms approach

In [10]:
import random
import time

In [11]:
def reproduce(indA, indB):
    idx = np.random.choice(range(1, indA.n))
    child = EightQueensState( list(indA.state[:idx])+list(indA.state[idx:]), n=indA.n )
    return child

def Gen_Algo_0(population,stop_rounds=80000,MUTATION_RATE = 0.05,timeout = 30):
    
    #set timeout = 30 seconds as specified in the requirements
    counter = 0 
    end_state = False
    
    ## set population
    while end_state == False and counter < stop_rounds:
        
         ## check if solution is in the population
        for i in range (len((population))):
            if population[i].is_not_goal() == False:
                end_state = True
                print(f'Goalllll, Step No: {counter}. The answer is: {population[i]}')  
                return
            
        new_population = []
        for i in range(len(population)):
            z = random.sample(population, 2)
            x = z[0]
            y = z[1]
            r = reproduce(x,y) # cross over x and y

            ## mutation
            for j in range(len(population[i].state)):
                if random.random() < MUTATION_RATE:
                    s = np.random.randint(len(population[i].state))
                    k = r.copy_replace(r.state,j,s)
                    r = EightQueensState(k)
                else:
                    continue

            new_population.append(r)

        if end_state == True:
            break

        population = new_population
        counter +=1
        
    print("No Solution within the timeframe")
    return 

#### Population size = 55

In [13]:
population_size=55
Queensize = 8
n = 20

avg_start = time.time()
for i in range(n):
    start_time = time.time()
    EightQueens = [EightQueensState(n=Queensize) for k in range(population_size)]
    Gen_Algo_0(EightQueens)
    end_time = time.time()
    print( f"It takes: {(end_time-start_time)} seconds to solve this N-Queens problem")
#     print( f"Average time per problem: {(end_time-start_time)} seconds")

avg_end = time.time()
print( f"Average time per problem: {(avg_end-avg_start)/n} seconds")

Goalllll, Step No: 2954. The answer is: Goal state! [2, 5, 7, 0, 3, 6, 4, 1]
It takes: 4.040396690368652 seconds to solve this N-Queens problem
Goalllll, Step No: 392. The answer is: Goal state! [3, 0, 4, 7, 5, 2, 6, 1]
It takes: 0.4787609577178955 seconds to solve this N-Queens problem
Goalllll, Step No: 1624. The answer is: Goal state! [6, 0, 2, 7, 5, 3, 1, 4]
It takes: 2.0855178833007812 seconds to solve this N-Queens problem
Goalllll, Step No: 5522. The answer is: Goal state! [3, 1, 4, 7, 5, 0, 2, 6]
It takes: 7.9645469188690186 seconds to solve this N-Queens problem
Goalllll, Step No: 1093. The answer is: Goal state! [7, 1, 3, 0, 6, 4, 2, 5]
It takes: 1.4485836029052734 seconds to solve this N-Queens problem
Goalllll, Step No: 4000. The answer is: Goal state! [0, 4, 7, 5, 2, 6, 1, 3]
It takes: 5.2918219566345215 seconds to solve this N-Queens problem
Goalllll, Step No: 2914. The answer is: Goal state! [2, 5, 7, 0, 3, 6, 4, 1]
It takes: 4.226235389709473 seconds to solve this N-Que

#### Population size = 58

In [14]:
population_size=58
Queensize = 8
n = 20

avg_start = time.time()
for i in range(n):
    start_time = time.time()
    EightQueens = [EightQueensState(n=Queensize) for k in range(population_size)]
    Gen_Algo_0(EightQueens)
    end_time = time.time()
    print( f"It takes: {(end_time-start_time)} seconds to solve this N-Queens problem")
#     print( f"Average time per problem: {(end_time-start_time)} seconds")

avg_end = time.time()
print( f"Average time per problem: {(avg_end-avg_start)/n} seconds")

Goalllll, Step No: 37629. The answer is: Goal state! [3, 1, 6, 2, 5, 7, 0, 4]
It takes: 52.29648184776306 seconds to solve this N-Queens problem
Goalllll, Step No: 3305. The answer is: Goal state! [6, 2, 0, 5, 7, 4, 1, 3]
It takes: 4.884356498718262 seconds to solve this N-Queens problem
Goalllll, Step No: 3766. The answer is: Goal state! [4, 6, 1, 3, 7, 0, 2, 5]
It takes: 5.556166648864746 seconds to solve this N-Queens problem
Goalllll, Step No: 1462. The answer is: Goal state! [2, 5, 7, 0, 3, 6, 4, 1]
It takes: 2.6284570693969727 seconds to solve this N-Queens problem
Goalllll, Step No: 4945. The answer is: Goal state! [5, 1, 6, 0, 2, 4, 7, 3]
It takes: 8.103738069534302 seconds to solve this N-Queens problem
Goalllll, Step No: 5888. The answer is: Goal state! [5, 2, 0, 6, 4, 7, 1, 3]
It takes: 9.807533740997314 seconds to solve this N-Queens problem
Goalllll, Step No: 15629. The answer is: Goal state! [1, 5, 7, 2, 0, 3, 6, 4]
It takes: 26.018595457077026 seconds to solve this N-Que

#### Population size = 200

In [15]:
population_size=200
Queensize = 8
n = 20

avg_start = time.time()
for i in range(n):
    start_time = time.time()
    EightQueens = [EightQueensState(n=Queensize) for k in range(population_size)]
    Gen_Algo_0(EightQueens)
    end_time = time.time()
    print( f"It takes: {(end_time-start_time)} seconds to solve this N-Queens problem")
#     print( f"Average time per problem: {(end_time-start_time)} seconds")

avg_end = time.time()
print( f"Average time per problem: {(avg_end-avg_start)/n} seconds")

Goalllll, Step No: 13699. The answer is: Goal state! [3, 1, 4, 7, 5, 0, 2, 6]
It takes: 65.55611371994019 seconds to solve this N-Queens problem
Goalllll, Step No: 8078. The answer is: Goal state! [5, 1, 6, 0, 2, 4, 7, 3]
It takes: 48.65571165084839 seconds to solve this N-Queens problem
Goalllll, Step No: 4523. The answer is: Goal state! [5, 7, 1, 3, 0, 6, 4, 2]
It takes: 35.11315631866455 seconds to solve this N-Queens problem
Goalllll, Step No: 2979. The answer is: Goal state! [2, 5, 7, 0, 4, 6, 1, 3]
It takes: 24.447218894958496 seconds to solve this N-Queens problem
Goalllll, Step No: 2966. The answer is: Goal state! [2, 4, 1, 7, 5, 3, 6, 0]
It takes: 23.690426349639893 seconds to solve this N-Queens problem
Goalllll, Step No: 526. The answer is: Goal state! [7, 1, 3, 0, 6, 4, 2, 5]
It takes: 4.360215425491333 seconds to solve this N-Queens problem
Goalllll, Step No: 5401. The answer is: Goal state! [4, 1, 7, 0, 3, 6, 2, 5]
It takes: 40.95479989051819 seconds to solve this N-Queen

#### Population size = 150

In [16]:
population_size=150
Queensize = 8
n = 20

avg_start = time.time()
for i in range(n):
    start_time = time.time()
    EightQueens = [EightQueensState(n=Queensize) for k in range(population_size)]
    Gen_Algo_0(EightQueens)
    end_time = time.time()
    print( f"It takes: {(end_time-start_time)} seconds to solve this N-Queens problem")
#     print( f"Average time per problem: {(end_time-start_time)} seconds")

avg_end = time.time()
print( f"Average time per problem: {(avg_end-avg_start)/n} seconds")

Goalllll, Step No: 3842. The answer is: Goal state! [4, 1, 5, 0, 6, 3, 7, 2]
It takes: 14.885345697402954 seconds to solve this N-Queens problem
Goalllll, Step No: 5347. The answer is: Goal state! [6, 2, 7, 1, 4, 0, 5, 3]
It takes: 21.351425647735596 seconds to solve this N-Queens problem
Goalllll, Step No: 17786. The answer is: Goal state! [4, 2, 0, 5, 7, 1, 3, 6]
It takes: 94.4757571220398 seconds to solve this N-Queens problem
Goalllll, Step No: 502. The answer is: Goal state! [1, 5, 0, 6, 3, 7, 2, 4]
It takes: 3.015359878540039 seconds to solve this N-Queens problem
Goalllll, Step No: 835. The answer is: Goal state! [5, 1, 6, 0, 2, 4, 7, 3]
It takes: 5.142454624176025 seconds to solve this N-Queens problem
Goalllll, Step No: 2429. The answer is: Goal state! [5, 3, 6, 0, 2, 4, 1, 7]
It takes: 14.974979162216187 seconds to solve this N-Queens problem
Goalllll, Step No: 5085. The answer is: Goal state! [2, 5, 3, 0, 7, 4, 6, 1]
It takes: 31.298381567001343 seconds to solve this N-Queen

### Version 2 - Roulette Wheel Selection - Select parents based on probability 

In [32]:
def reproduce(indA, indB):
    idx = np.random.choice(range(1, indA.n))
    child = EightQueensState( list(indA.state[:idx])+list(indA.state[idx:]), n=indA.n )
    return child

def choose_best(population, n = -1):
    sorted_population = sorted(population,key=lambda ind: ind.cost()) # sort the cost in acc order 
    if n != -1:
        sorted_population = sorted_population[:n]
    fitness = [1/ind.cost() for ind in sorted_population]
    total_fitness = sum(fitness)
    prob = [f/total_fitness for f in fitness]
    return sorted_population, prob

def Roulette(population, prob):
    selected_parents = []
    for i in range(2):
        r =  np.random.random()
        total_prob = 0
        for j in range(len(prob)):
            total_prob += prob[i]
            if r <= total_prob:
                selected_parents.append(population[j])
    return selected_parents

def containsGoal(population):
    return [ind for ind in population if ind.is_goal()]

def Gen_Algo_9(population,stop_rounds=50000,MUTATION_RATE = 0.05,timeout = 30):
    counter = 0 
    end_state = False
    
    ## set population
    while end_state == False and counter < stop_rounds:
        solved_ind = containsGoal(population)
        if len(solved_ind) > 0:
            print(f'Goalllll, Step No: {counter}. The answer is: {solved_ind[0]}')  
            return
        
        new_population = []
        best_ind, best_prob = choose_best(population)#, int(len(population) * .8))
        for i in range(len(population)):
            chosen_parents = Roulette(best_ind, best_prob)
            r = reproduce(chosen_parents[0],chosen_parents[1]) # cross over x and y

            ## mutation
            Queensize = len(population[i].state)
            if random.random() < MUTATION_RATE:
                j = np.random.randint(Queensize)
                s = np.random.randint(Queensize)
                r = EightQueensState(r.copy_replace(r.state,j,s))
                
            new_population.append(r)
        population = new_population
        counter +=1
        
    print("No Solution within the timeframe")
    return 

In [33]:
population_size=10
Queensize = 8
n = 20

avg_start = time.time()
for i in range(n):
    start_time = time.time()
    EightQueens = [EightQueensState(n=Queensize) for k in range(population_size)]
    Gen_Algo_9(EightQueens)
    end_time = time.time()
    print( f"It takes: {(end_time-start_time)} seconds to solve this N-Queens problem")
#     print( f"Average time per problem: {(end_time-start_time)} seconds")

avg_end = time.time()
print( f"Average time per problem: {(avg_end-avg_start)/n} seconds")

Goalllll, Step No: 650. The answer is: Goal state! [3, 5, 7, 1, 6, 0, 2, 4]
It takes: 0.18358159065246582 seconds to solve this N-Queens problem
Goalllll, Step No: 6116. The answer is: Goal state! [3, 1, 6, 2, 5, 7, 0, 4]
It takes: 1.2690086364746094 seconds to solve this N-Queens problem
Goalllll, Step No: 428. The answer is: Goal state! [4, 1, 3, 5, 7, 2, 0, 6]
It takes: 0.10738801956176758 seconds to solve this N-Queens problem
Goalllll, Step No: 15028. The answer is: Goal state! [3, 1, 7, 5, 0, 2, 4, 6]
It takes: 2.8198013305664062 seconds to solve this N-Queens problem
Goalllll, Step No: 2993. The answer is: Goal state! [5, 2, 0, 7, 3, 1, 6, 4]
It takes: 0.6501362323760986 seconds to solve this N-Queens problem
Goalllll, Step No: 108. The answer is: Goal state! [3, 5, 7, 1, 6, 0, 2, 4]
It takes: 0.04152536392211914 seconds to solve this N-Queens problem
Goalllll, Step No: 2955. The answer is: Goal state! [4, 7, 3, 0, 6, 1, 5, 2]
It takes: 0.6463611125946045 seconds to solve this N

----

### Version 3 - Always choose the best parents

In [19]:
def reproduce(indA, indB):
    idx = np.random.choice(range(1, indA.n))
    child = EightQueensState( list(indA.state[:idx])+list(indA.state[idx:]), n=indA.n )
    return child

def choose_best(population):
    scores=[]
    for ind in population:
        scores.append(ind.cost())
    idx = np.argmin(np.array(scores))
    return population[idx], scores[idx]==0
        
def tournament(population):
    random.shuffle(population)
    x, solved = choose_best(population[:len(population)//2])
    y, solved_2 = choose_best(population[len(population)//2:])
    return x,y, solved or solved_2
    
def Gen_Algo(population,stop_rounds=55000,MUTATION_RATE = 0.05,timeout = 30):
    #set timeout = 30 seconds as specified in the requirements
    counter = 0 
    end_state = False
    
    ## set population
    while end_state == False and counter < stop_rounds:
        new_population = []
        for i in range(len(population)):
            x,y,solved = tournament(population)
            if solved:
                end_state = True
                if x.is_goal():
                    print(f'Goalllll, Step No: {counter}. The answer is: {x}')   
                else:
                    print(f'Goalllll, Step No: {counter}. The answer is: {y}')  
                return
                         
            r = reproduce(x,y) # cross over x and y

            ## mutation
            Queensize = len(population[i].state)
            for j in range(Queensize):
                if random.random() < MUTATION_RATE:
                    s = np.random.randint(len(population[i].state))
                    r = EightQueensState(r.copy_replace(r.state,j,s))
                else:
                    continue

            new_population.append(r)

        population = new_population
        counter +=1
        
    print("No Solution within the timeframe")
    return 

In [20]:
population_size=5
Queensize = 8
n = 20

avg_start = time.time()
for i in range(n):
    start_time = time.time()
    EightQueens = [EightQueensState(n=Queensize) for k in range(population_size)]
    Gen_Algo(EightQueens)
    end_time = time.time()
    print( f"It takes: {(end_time-start_time)} seconds to solve this N-Queens problem")
#     print( f"Average time per problem: {(end_time-start_time)} seconds")

avg_end = time.time()
print( f"Average time per problem: {(avg_end-avg_start)/n} seconds")

Goalllll, Step No: 612. The answer is: Goal state! [6, 3, 1, 4, 7, 0, 2, 5]
It takes: 0.23773646354675293 seconds to solve this N-Queens problem
Goalllll, Step No: 1570. The answer is: Goal state! [4, 2, 7, 3, 6, 0, 5, 1]
It takes: 0.5351166725158691 seconds to solve this N-Queens problem
Goalllll, Step No: 896. The answer is: Goal state! [4, 0, 7, 3, 1, 6, 2, 5]
It takes: 0.3059422969818115 seconds to solve this N-Queens problem
Goalllll, Step No: 1087. The answer is: Goal state! [6, 2, 7, 1, 4, 0, 5, 3]
It takes: 0.29280519485473633 seconds to solve this N-Queens problem
Goalllll, Step No: 996. The answer is: Goal state! [4, 2, 0, 5, 7, 1, 3, 6]
It takes: 0.3378634452819824 seconds to solve this N-Queens problem
Goalllll, Step No: 237. The answer is: Goal state! [4, 1, 7, 0, 3, 6, 2, 5]
It takes: 0.0695180892944336 seconds to solve this N-Queens problem
Goalllll, Step No: 2515. The answer is: Goal state! [0, 6, 4, 7, 1, 3, 5, 2]
It takes: 0.7695944309234619 seconds to solve this N-Qu

---


### Version 4 - Adaptive Mutation


#### higher mutation rate when the cost is lower (i.e. better chromosome) 


In [21]:
def reproduce(indA, indB):
    idx = np.random.choice(range(1, indA.n))
    child = EightQueensState( list(indA.state[:idx])+list(indA.state[idx:]), n=indA.n )
    return child

def choose_best(population):
    #print(population)
    scores=[]
    for ind in population:
        scores.append(ind.cost())
    idx = np.argmin(np.array(scores))
    return population[idx], scores[idx]==0
        
def tournament(population):
    random.shuffle(population)
    x, solved = choose_best(population[:len(population)//2])
    y, solved_2 = choose_best(population[len(population)//2:])
    return x,y, solved or solved_2

def Gen_Algo2(population,stop_rounds=25000,MUTATION_RATE = 0.03,timeout = 30):
    #set timeout = 30 seconds as specified in the requirements
    counter = 0 
    end_state = False
    
    ## set population
    while end_state == False and counter < stop_rounds:
        new_population = []
        for i in range(len(population)):
            x,y,solved = tournament(population)
            if solved:
                end_state = True
                if x.is_goal():
                    print(f'Goalllll, Step No: {counter}. The answer is: {x}')   
                else:
                    print(f'Goalllll, Step No: {counter}. The answer is: {y}')  
                return
                         
            r = reproduce(x,y) # cross over x and y
            
            if r.cost() > len(population[i].state) / 2:
                MUTATION_RATE = 0.08
            else:
                MUTATION_RATE = 0.03

            ## mutation
            Queensize = len(population[i].state)
            for j in range(Queensize):
                if random.random() < MUTATION_RATE:
                    s = np.random.randint(len(population[i].state))
                    k = r.copy_replace(r.state,j,s)
                    r = EightQueensState(k)
                
                else:
                    continue

            new_population.append(r)

        population = new_population
        counter +=1
        
    print("No Solution within the timeframe")
    return 

In [22]:
population_size=5
Queensize = 10
n = 20

avg_start = time.time()
for i in range(n):
    start_time = time.time()
    EightQueens = [EightQueensState(n=Queensize) for k in range(population_size)]
    Gen_Algo2(EightQueens)
    end_time = time.time()
    print( f"It takes: {(end_time-start_time)} seconds to solve this N-Queens problem")
#     print( f"Average time per problem: {(end_time-start_time)} seconds")

avg_end = time.time()
print( f"Average time per problem: {(avg_end-avg_start)/n} seconds")

Goalllll, Step No: 4081. The answer is: Goal state! [3, 6, 2, 5, 1, 9, 0, 8, 4, 7]
It takes: 1.5753531455993652 seconds to solve this N-Queens problem
Goalllll, Step No: 79. The answer is: Goal state! [0, 8, 5, 1, 6, 9, 2, 4, 7, 3]
It takes: 0.0342254638671875 seconds to solve this N-Queens problem
Goalllll, Step No: 5061. The answer is: Goal state! [7, 2, 8, 3, 9, 0, 5, 1, 4, 6]
It takes: 1.7688112258911133 seconds to solve this N-Queens problem
Goalllll, Step No: 5171. The answer is: Goal state! [7, 2, 4, 1, 9, 0, 5, 3, 8, 6]
It takes: 1.7841522693634033 seconds to solve this N-Queens problem
Goalllll, Step No: 5953. The answer is: Goal state! [1, 4, 8, 0, 9, 3, 6, 2, 7, 5]
It takes: 2.0089290142059326 seconds to solve this N-Queens problem
Goalllll, Step No: 2479. The answer is: Goal state! [5, 9, 6, 1, 3, 8, 0, 7, 4, 2]
It takes: 0.8935985565185547 seconds to solve this N-Queens problem
Goalllll, Step No: 2348. The answer is: Goal state! [5, 3, 8, 0, 2, 6, 9, 7, 1, 4]
It takes: 0.8

---

### Version 5 - Uniform Selection

Conduct the crossover on the gene level instead of Chromose level 

In [27]:
def uniformCrossover(indA, indB):
    child_list = []
    stateA = indA.state
    stateB = indB.state
    
    for i in range(indA.n):
        if random.random() < 0.5:
            tempA = stateA[i]
            stateA[i] = stateB[i]
            stateB[i] = tempA
            
    child = EightQueensState(indA.state)   
    return child 

def choose_best(population):
    #print(population)
    scores=[]
    for ind in population:
        scores.append(ind.cost())
    idx = np.argmin(np.array(scores))
    return population[idx], scores[idx]==0
        
def tournament(population):
    random.shuffle(population)
    x, solved = choose_best(population[:len(population)//2])
    y, solved_2 = choose_best(population[len(population)//2:])
    return x,y, solved or solved_2

def Gen_Algo5(population,stop_rounds=25000,MUTATION_RATE = 0.03,timeout = 30):
    counter = 0 
    end_state = False
    
    ## set population
    while end_state == False and counter < stop_rounds:
        new_population = []
        for i in range(len(population)):
            x,y,solved = tournament(population)
            if solved:
                end_state = True
                if x.is_goal():
                    print(f'Goalllll, Step No: {counter}. The answer is: {x}')   
                else:
                    print(f'Goalllll, Step No: {counter}. The answer is: {y}')  
                return
                         
            r = uniformCrossover(x,y) # cross over x and y
            
            if r.cost() > len(population[i].state) / 2:
                MUTATION_RATE = 0.08
            else:
                MUTATION_RATE = 0.03

            ## mutation
            Queensize = len(population[i].state)
            for j in range(Queensize):
                if random.random() < MUTATION_RATE:
                    s = np.random.randint(len(population[i].state))
                    k = r.copy_replace(r.state,j,s)
                    r = EightQueensState(k)
                
                else:
                    continue

            new_population.append(r)

        population = new_population
        counter +=1
        
    print("No Solution within the timeframe")
    return 

In [29]:
population_size=5
Queensize = 10
n = 20

avg_start = time.time()
for i in range(n):
    start_time = time.time()
    EightQueens = [EightQueensState(n=Queensize) for k in range(population_size)]
    Gen_Algo5(EightQueens)
    end_time = time.time()
    print( f"It takes: {(end_time-start_time)} seconds to solve this N-Queens problem")
#     print( f"Average time per problem: {(end_time-start_time)} seconds")

avg_end = time.time()
print( f"Average time per problem: {(avg_end-avg_start)/n} seconds")

Goalllll, Step No: 882. The answer is: Goal state! [5 9 2 6 3 1 8 4 0 7]
It takes: 1.523324966430664 seconds to solve this N-Queens problem
Goalllll, Step No: 1950. The answer is: Goal state! [5 7 4 1 3 9 6 8 2 0]
It takes: 3.3345160484313965 seconds to solve this N-Queens problem
Goalllll, Step No: 4904. The answer is: Goal state! [3 0 7 5 1 9 6 8 2 4]
It takes: 7.980743169784546 seconds to solve this N-Queens problem
Goalllll, Step No: 1161. The answer is: Goal state! [3 7 2 4 8 0 5 9 6 1]
It takes: 2.0097579956054688 seconds to solve this N-Queens problem
Goalllll, Step No: 3552. The answer is: Goal state! [5 1 9 7 3 8 0 2 4 6]
It takes: 6.132277011871338 seconds to solve this N-Queens problem
Goalllll, Step No: 3640. The answer is: Goal state! [1 4 7 5 2 9 6 0 3 8]
It takes: 6.381232500076294 seconds to solve this N-Queens problem
Goalllll, Step No: 1518. The answer is: Goal state! [3 1 8 5 9 6 0 2 4 7]
It takes: 2.690034866333008 seconds to solve this N-Queens problem
Goalllll, St